# Chilbolton Source 3 and Source 4 Inversion
-------------------------------

This notebook was used to produce the Source 3 and Source 4 parameter estimation results in Section 5 and Supplementary Material B 2.7. These are presented in the notebook: "Chilbolton sources inversion results.ipynb".

<div class="alert alert-block alert-warning">
<b>PACKAGE REQUIREMENT:</b> Package "sourceinversion". Install using:<br>
pip install -q sourceinversion
</div>

In [1]:
# pip install -q sourceinversion

<div class="alert alert-block alert-info">
<b>DATA:</b> Need to replace data file paths with your own local path. The files are all located in the folder:<br>
Paper 1: Code/Data/...
</div>

<br><br><br><br><br>

### Importing "sourceinversion" Package

In [2]:
import sourceinversion.atmospheric_measurements as gp
import sourceinversion.mcmc as mcmc

2024-07-23 13:27:51.429565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Libraries

In [3]:
import pickle
import pandas as pd
import numpy as np
from jax.flatten_util import ravel_pytree
import jax.numpy as jnp
from jax import config
import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions
import itertools
from jax import lax
from pyDOE import lhs

config.update("jax_enable_x64", True)


<br><br><br><br><br>

## 1.  Chilbolton Data

<div class="alert alert-block alert-info">
<b>DATA:</b> Need to replace data file paths with your own local path. The files are all located in the folder:<br>
Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_3_and_4/Chilbolton_CH4_measurements_source_3and4.pkl
</div>

1.1 CH4 measurements

In [4]:
with open('/home/newmant1/PhD/Packages/Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_3_and_4/Chilbolton_CH4_measurements_source_3and4.pkl', 'rb') as f:
    observations = pickle.load(f)

print(observations)
data = observations.values

      Measurements
0         2.156402
1         2.323252
2         2.695396
3         2.180471
4         2.181575
...            ...
1644      2.077906
1645      2.079868
1646      2.114129
1647      2.995524
1648      2.982547

[1649 rows x 1 columns]


1.2 Wind field and rolling standard deviation of the horizontal and vertical wind direction.

<div class="alert alert-block alert-info">
<b>DATA:</b> Need to replace data file paths with your own local path. The files are all located in the folder:<br>
Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_3_and_4/Chilbolton_windfield_source_3and4.pkl
</div>

In [5]:
with open('/home/newmant1/PhD/Packages/Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_3_and_4/Chilbolton_windfield_source_3and4.pkl', 'rb') as f:
    tangamma_ts = pickle.load(f)
    wind_field = tangamma_ts[['Average Speed', 'Average Direction']]
    
print(wind_field)

     Average Speed  Average Direction
0         2.148621         161.316738
1         2.742933         147.034253
2         2.670685         152.610808
3         2.793845         166.890524
4         3.088895         158.469113
..             ...                ...
235       3.073157          66.605103
236       3.270097          69.169916
237       4.056264          77.726880
238       4.371059          82.811326
239       5.100623          85.347159

[240 rows x 2 columns]


Removing release_17 data: 
The inversion can be made with release_17 data or without (do not run the code below if wishing to include release_17). All release_17 beam 2 observations are corrupt and therefore were removed during the data processing. Release_17 therefore contains measurements for all beams apart from beam 2.

In [6]:
wind_field = wind_field.iloc[0:209]
print(wind_field)

# Create a grouping key
group_key = observations.index // 240

# Select the first 209 (=240-31) rows from each group
df_new = observations.groupby(group_key).head(209)
observations = df_new

print(observations)
data = observations.values

     Average Speed  Average Direction
0         2.148621         161.316738
1         2.742933         147.034253
2         2.670685         152.610808
3         2.793845         166.890524
4         3.088895         158.469113
..             ...                ...
204       3.853372         125.586786
205       4.038335         124.080517
206       4.336893         128.099723
207       4.898623         146.980776
208       4.857069         148.027751

[209 rows x 2 columns]
      Measurements
0         2.156402
1         2.323252
2         2.695396
3         2.180471
4         2.181575
...            ...
1644      2.077906
1645      2.079868
1646      2.114129
1647      2.995524
1648      2.982547

[1463 rows x 1 columns]


1.3 Sensor layout

<div class="alert alert-block alert-info">
<b>DATA:</b> Need to replace data file paths with your own local path. The files are all located in the folder:<br>
Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Sensor_reflector_locations/Chilbolton_instruments_location.pkl
</div>

In [7]:
with open('/home/newmant1/PhD/Packages/Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Sensor_reflector_locations/Chilbolton_instruments_location.pkl', 'rb') as f:
    instruments_location = pickle.load(f)

creating integration points along beam every 0.40 meters

In [8]:
number_of_point_sensors = {
    "reflector_1": 18*5,
    "reflector_2": 33*5,
    "reflector_3": 22*5,
    "reflector_4": 49*5,
    "reflector_5": 42*5,
    "reflector_6": 29*5,
    "reflector_7": 17*5,
}

def get_equally_spaced_points(point1, point2, num_points):
    # Calculate the step size for each dimension
    step_size = [(p2 - p1) / (num_points - 1) for p1, p2 in zip(point1, point2)]

    # Calculate the coordinates of the equally spaced points
    points = [[p1 + i * step for p1, step in zip(point1, step_size)] for i in range(num_points)]

    return points

point_sensors_1_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_1"], number_of_point_sensors["reflector_1"])
point_sensors_2_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_2"], number_of_point_sensors["reflector_2"])
point_sensors_3_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_3"], number_of_point_sensors["reflector_3"])
point_sensors_4_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_4"], number_of_point_sensors["reflector_4"])
point_sensors_5_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_5"], number_of_point_sensors["reflector_5"])
point_sensors_6_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_6"], number_of_point_sensors["reflector_6"])
point_sensors_7_location = get_equally_spaced_points(instruments_location["line_of_sight_sensor"], instruments_location["reflector_7"], number_of_point_sensors["reflector_7"])

1.4 Sources

<div class="alert alert-block alert-info">
<b>DATA:</b> Need to replace data file paths with your own local path. The files are all located in the folder:<br>
Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_locations_and_emission_rates/Chilbolton_sources_locations_and_emission_rates.pkl
</div>

In [9]:
with open('/home/newmant1/PhD/Packages/Paper 1: Code/Data/Chilbolton_data_files/Postprocessed/Source_locations_and_emission_rates/Chilbolton_sources_locations_and_emission_rates.pkl', 'rb') as f:
    sources = pickle.load(f)

<br><br><br><br><br>

## 2. Setting up for Inversion using "sourceinversion" package


<div class="alert alert-block alert-info">
<b>Note:</b> Variables set to "None" are used only when simulating gas emissions.
</div>

In [10]:
# Grid specification based on the Chilbolton terrain dimensions
# (used grid Search parameter initialisation or grid-based inversion)
grid = gp.Grid(
    x_range = (jnp.array(40.0), jnp.array(80.0)), 
    y_range = (jnp.array(0.0), jnp.array(110.0)),
    z_range= (jnp.array(0.0), jnp.array(0.0)),
    dx = jnp.array(10),
    dy = jnp.array(10),
    dz = jnp.array(1),
)


# Source 3 and Source 4 location
source_location = gp.SourceLocation(
    source_location_x = jnp.array([sources["source_3_location"][0], sources["source_4_location"][0]]),
    source_location_y = jnp.array([sources["source_3_location"][1], sources["source_4_location"][1]]),
    source_location_z = jnp.array([sources["source_3_location"][2], sources["source_4_location"][2]]),
)


# Atmospheric State
atmospheric_state = gp.AtmosphericState(
    emission_rate = jnp.array([sources["source_3_emission_rate"], \
                            sources["source_4_emission_rate"]]),              
    source_half_width = jnp.array(1.0),                                 # Source is a square of 2m side length
    max_abl = jnp.array(1000.0),
    background_mean = None,                                  
    background_std = None,       
    background_seed = None,
    background_filter = None,        
    Gaussian_filter_kernel = None,              
    horizontal_opening_angle= None,
    vertical_opening_angle = None,
    a_horizontal = None,
    a_vertical = None,          
    b_horizontal = None,         
    b_vertical = None,        
)

# Sensor layout
def flatten_list_of_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

sensors_settings =  gp.SensorsSettings(
    layout = None,
    sensor_number = jnp.array(7),
    measurement_error_var = None,
    sensor_seed = None,
    measurement_error_seed = None,
    sensor_locations =  flatten_list_of_lists([point_sensors_1_location, point_sensors_2_location, point_sensors_3_location, point_sensors_4_location, point_sensors_5_location, point_sensors_6_location, point_sensors_7_location]), 
)


# Gaussian Plume model
gaussianplume = gp.GaussianPlume(grid, source_location, wind_field, atmospheric_state, sensors_settings)

fixed  = gaussianplume.fixed_objects_of_gridfree_chilbolton_coupling_matrix(simulation = False, wind_direction=wind_field["Average Direction"].values, wind_speed=wind_field["Average Speed"].values, tangamma_ts = tangamma_ts, number_of_time_steps=wind_field.shape[0], source3and4=True, release_17=False)
fixed_ref1 = fixed[0], fixed[7], fixed[14], fixed[15], fixed[35], fixed[36], fixed[16], fixed[37], fixed[44]
fixed_ref2 = fixed[1], fixed[8], fixed[17], fixed[18], fixed[35], fixed[36], fixed[19], fixed[38], fixed[45]
fixed_ref3 = fixed[2], fixed[9], fixed[20], fixed[21], fixed[35], fixed[36], fixed[22], fixed[39], fixed[46]
fixed_ref4 = fixed[3], fixed[10], fixed[23], fixed[24], fixed[35], fixed[36], fixed[25], fixed[40], fixed[47]
fixed_ref5 = fixed[4], fixed[11], fixed[26], fixed[27], fixed[35], fixed[36], fixed[28], fixed[41], fixed[48]
fixed_ref6 = fixed[5], fixed[12], fixed[29], fixed[30], fixed[35], fixed[36], fixed[31], fixed[42], fixed[49]
fixed_ref7 = fixed[6], fixed[13], fixed[32], fixed[33], fixed[35], fixed[36], fixed[34], fixed[43], fixed[50]


# Parameter priors
priors = mcmc.Priors(
    # Slab allocation rate prior (used in grid-based inversion)
    theta = 0.1,

    # Emission rate (log(s)): Log scale Slab and spike prior (used in grid-based inversion)
    log_spike_mean = -25.0,
    log_spike_var = 10.0,
    log_slab_mean = -7.5,
    log_slab_var = 1.5,

    # Source location (x,y):
    source_location_x_mean = 50.0,
    source_location_x_var = 25.0,
    source_location_y_mean = 50.0,
    source_location_y_var = 25.0,

    # Measurement error variance (sigma squared)
    sigma_squared_con = 1e-11,
    sigma_squared_rate = 1e-8,

    # Background gas concentration (beta)
    mean_background_prior = 1.92,
    variance_background_prior = 0.1**2,

    # Dispersion parameter (a_H, a_V, b_H, b_V)
    a_mean = jnp.log(0.6),
    a_var = 0.5**2,
    b_mean = jnp.log(0.6),
    b_var = 0.2**2,
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


<br><br><br><br><br>

## 3. Source 3 and Source 4 Parameter Estimation

### 3.1 Grid Search

Here we only estimate source emission rate and location while fixing background gas concentration, measurement error variance and dispersion parameters.

In [11]:
emission_granulaty_in_kg_per_s = 0.001/3

s_range = np.arange(0, 0.001, emission_granulaty_in_kg_per_s)
x_range = grid.x
y_range = grid.y

def get_ranges_combinations(x, y):
    xx, yy = np.meshgrid(x, y)
    combinations = np.vstack([xx.ravel(), yy.ravel()]).T
    return combinations

ranges = []
ranges.append(get_ranges_combinations(s_range,s_range))
ranges.append(get_ranges_combinations(x_range,x_range))
ranges.append(get_ranges_combinations(y_range,y_range))

ranges

[array([[0.        , 0.        ],
        [0.00033333, 0.        ],
        [0.00066667, 0.        ],
        [0.        , 0.00033333],
        [0.00033333, 0.00033333],
        [0.00066667, 0.00033333],
        [0.        , 0.00066667],
        [0.00033333, 0.00066667],
        [0.00066667, 0.00066667]]),
 array([[40., 40.],
        [50., 40.],
        [60., 40.],
        [70., 40.],
        [80., 40.],
        [40., 50.],
        [50., 50.],
        [60., 50.],
        [70., 50.],
        [80., 50.],
        [40., 60.],
        [50., 60.],
        [60., 60.],
        [70., 60.],
        [80., 60.],
        [40., 70.],
        [50., 70.],
        [60., 70.],
        [70., 70.],
        [80., 70.],
        [40., 80.],
        [50., 80.],
        [60., 80.],
        [70., 80.],
        [80., 80.]]),
 array([[  0.,   0.],
        [ 10.,   0.],
        [ 20.,   0.],
        [ 30.,   0.],
        [ 40.,   0.],
        [ 50.,   0.],
        [ 60.,   0.],
        [ 70.,   0.],
        [ 80.,

In [12]:
def search_likelihood(s, x, y, initial_sgm, initial_betas, scheme, stability_class, number_time_steps, release_17=False):
    """
    Returns the positive log posterior of the point sensors measurements model. 

    """
    if stability_class == False:
        stability_class = None
        estimated = True
    else:
        estimated = False
        
    coupling_matrix_ref1 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref1, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref2 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref2, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref3 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref3, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref4 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref4, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref5 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref5, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref6 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref6, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref7 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref7, x, y, None, 1.0, 1.0, 1.0, 1.0, simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    
    if release_17:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_time_steps,17*5, order='F')
    else:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_time_steps,17*5, order='F')

    path_averaged_coupling_matrix_ref1_src1 = reshaped_coupling_matrix_ref1_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref1_src2 = reshaped_coupling_matrix_ref1_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src1 = reshaped_coupling_matrix_ref2_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src2 = reshaped_coupling_matrix_ref2_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src1 = reshaped_coupling_matrix_ref3_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src2 = reshaped_coupling_matrix_ref3_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src1 = reshaped_coupling_matrix_ref4_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src2 = reshaped_coupling_matrix_ref4_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src1 = reshaped_coupling_matrix_ref5_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src2 = reshaped_coupling_matrix_ref5_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src1 = reshaped_coupling_matrix_ref6_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src2 = reshaped_coupling_matrix_ref6_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src1 = reshaped_coupling_matrix_ref7_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src2 = reshaped_coupling_matrix_ref7_src2.mean(axis=1)
    
    source_1_path_averaged_A = [path_averaged_coupling_matrix_ref1_src1, path_averaged_coupling_matrix_ref2_src1, path_averaged_coupling_matrix_ref3_src1, path_averaged_coupling_matrix_ref4_src1, path_averaged_coupling_matrix_ref5_src1, path_averaged_coupling_matrix_ref6_src1, path_averaged_coupling_matrix_ref7_src1]
    source_2_path_averaged_A = [path_averaged_coupling_matrix_ref1_src2, path_averaged_coupling_matrix_ref2_src2, path_averaged_coupling_matrix_ref3_src2, path_averaged_coupling_matrix_ref4_src2, path_averaged_coupling_matrix_ref5_src2, path_averaged_coupling_matrix_ref6_src2, path_averaged_coupling_matrix_ref7_src2]
    
    if release_17 == False:
        source_1_A = jnp.array(source_1_path_averaged_A).reshape(-1,1)
        source_2_A = jnp.array(source_2_path_averaged_A).reshape(-1,1)
        A = jnp.concatenate([source_1_A, source_2_A], axis=1)
        log_likelihood = tfd.Normal(loc = (jnp.matmul(A,s.reshape(-1,1)) + jnp.repeat(initial_betas, number_time_steps).reshape(-1,1)), \
                                    scale= jnp.sqrt(initial_sgm)).log_prob(data)
        log_posterior = jnp.sum(log_likelihood)
    else:
        source_1_A = jnp.array([item for sublist in source_1_path_averaged_A for item in sublist]).reshape((number_time_steps*6)+number_time_steps,2)
        source_2_A = jnp.array([item for sublist in source_2_path_averaged_A for item in sublist]).reshape((number_time_steps*6)+number_time_steps,2)
        A = jnp.concatenate([source_1_A, source_2_A], axis=1)
        log_likelihood = tfd.Normal(loc = (jnp.matmul(A,s.reshape(-1,1)) + jnp.repeat(initial_betas.mean(), (number_time_steps*6)+number_time_steps).reshape(-1,1)), \
                                    scale= jnp.sqrt(initial_sgm)).log_prob(data)

    return log_posterior

In [13]:
# Generate all combinations of parameters
sources_param = jnp.zeros(len(ranges)*2)
parameter_combinations = jnp.array(list(itertools.product(*ranges[:len(ranges)+1])))
print(f"Number of parameter combinations: {parameter_combinations.shape[0]}")

Number of parameter combinations: 32400


In [14]:
# Initial parameter values for:

# Sensor measurement error variance
initial_sgm = 1e-5
# Background gas concentration
initial_betas = jnp.repeat(1.92, 7)

# Step function
def one_step(_, parameters):
    s, x, y = parameters[0], parameters[1], parameters[2]
    new_likelihood = search_likelihood(s, x, y, initial_sgm, initial_betas, "Draxler", False, wind_field.shape[0], release_17=False)
    return new_likelihood, new_likelihood
# Use lax.scan to iterate over the parameter combinations
_, likelihoods = lax.scan(one_step, 0.0, parameter_combinations)
# Analysing output
likelihood_df = pd.DataFrame(likelihoods, columns = ['log_likelihood'])
parameters_df = pd.DataFrame(parameter_combinations.reshape(-1,6), columns = ["s1", "s2", "x1", "x2", "y1", "y2"])
max_likelihood_index = likelihood_df.idxmax()
max_likelihood_row = parameters_df.iloc[max_likelihood_index]
sources_param = sources_param.at[:].set(max_likelihood_row.values.flatten())
print(f"Grid search maximum likelihood estimation of emission rate and location:")
print(max_likelihood_row)

rate_1 = max_likelihood_row.values.flatten()[0]
rate_2 = max_likelihood_row.values.flatten()[1]
loc_x_1 = max_likelihood_row.values.flatten()[2]
loc_x_2 = max_likelihood_row.values.flatten()[3]
loc_y_1 = max_likelihood_row.values.flatten()[4]
loc_y_2 = max_likelihood_row.values.flatten()[5]

Grid search maximum likelihood estimation of emission rate and location:
             s1        s2    x1    x2    y1     y2
21580  0.000667  0.000333  80.0  80.0  40.0  100.0


<br>

### 3.2 Latin Hypercube

Here we estimate all parameters simultaneously.

In [15]:
def latin_hypercube_sampling(param_ranges, num_samples):
    """
    Generate a Latin Hypercube Sample within specified parameter ranges.

    Parameters:
    param_ranges (list of tuple): A list of tuples specifying the range (min, max) for each parameter.
    num_samples (int): The number of samples to generate.

    Returns:
    numpy.ndarray: A num_samples x num_params matrix where each column corresponds to a parameter and each row corresponds to a sample.
    """
    num_params = len(param_ranges)
    
    # Generate the Latin Hypercube Sample
    lhd = lhs(num_params, num_samples)
    
    # Scale the samples to be within the parameter ranges
    for i in range(num_params):
        min_val, max_val = param_ranges[i]
        lhd[:, i] = lhd[:, i] * (max_val - min_val) + min_val

    return lhd


def latin_hypercube_likelihood(s1, s2, x1, x2, y1, y2, sgm, a_h, a_v, b_h, b_v, scheme, stability_class, number_time_steps, release_17=False):
    """
    Returns the positive log posterior of the point sensors measurements model. 

    """
    if stability_class == False:
        stability_class = None
        estimated = True
    else:
        estimated = False

    coupling_matrix_ref1 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref1, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref2 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref2, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref3 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref3, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref4 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref4, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref5 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref5, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref6 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref6, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    coupling_matrix_ref7 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref7, [x1,x2], [y1,y2], None, jnp.array(a_h), jnp.array(a_v), jnp.array(b_h), jnp.array(b_v), simulation=False, estimated=estimated, scheme=scheme, stability_class=stability_class)
    
    if release_17:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_time_steps - 31,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_time_steps - 31,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_time_steps,17*5, order='F')
    else:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_time_steps,17*5, order='F')
    
    path_averaged_coupling_matrix_ref1_src1 = reshaped_coupling_matrix_ref1_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref1_src2 = reshaped_coupling_matrix_ref1_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src1 = reshaped_coupling_matrix_ref2_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src2 = reshaped_coupling_matrix_ref2_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src1 = reshaped_coupling_matrix_ref3_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src2 = reshaped_coupling_matrix_ref3_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src1 = reshaped_coupling_matrix_ref4_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src2 = reshaped_coupling_matrix_ref4_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src1 = reshaped_coupling_matrix_ref5_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src2 = reshaped_coupling_matrix_ref5_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src1 = reshaped_coupling_matrix_ref6_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src2 = reshaped_coupling_matrix_ref6_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src1 = reshaped_coupling_matrix_ref7_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src2 = reshaped_coupling_matrix_ref7_src2.mean(axis=1)

    source_1_path_averaged_A = [path_averaged_coupling_matrix_ref1_src1, path_averaged_coupling_matrix_ref2_src1, path_averaged_coupling_matrix_ref3_src1, path_averaged_coupling_matrix_ref4_src1, path_averaged_coupling_matrix_ref5_src1, path_averaged_coupling_matrix_ref6_src1, path_averaged_coupling_matrix_ref7_src1]
    source_2_path_averaged_A = [path_averaged_coupling_matrix_ref1_src2, path_averaged_coupling_matrix_ref2_src2, path_averaged_coupling_matrix_ref3_src2, path_averaged_coupling_matrix_ref4_src2, path_averaged_coupling_matrix_ref5_src2, path_averaged_coupling_matrix_ref6_src2, path_averaged_coupling_matrix_ref7_src2]
    
    if release_17 == False:
        source_1_A = jnp.array(source_1_path_averaged_A).reshape(-1,1)
        source_2_A = jnp.array(source_2_path_averaged_A).reshape(-1,1)
        A = jnp.concatenate([source_1_A, source_2_A], axis=1)
        log_likelihood = tfd.Normal(loc = (jnp.matmul(A,jnp.array([s1,s2]).reshape(-1,1)) + jnp.repeat(initial_betas, number_time_steps).reshape(-1,1)), \
                                    scale= jnp.sqrt(sgm)).log_prob(data)
        log_posterior = jnp.sum(log_likelihood)
    else:
        source_1_A = jnp.array([item for sublist in source_1_path_averaged_A for item in sublist]).reshape((number_time_steps*6)+number_time_steps,2)
        source_2_A = jnp.array([item for sublist in source_2_path_averaged_A for item in sublist]).reshape((number_time_steps*6)+number_time_steps,2)
        A = jnp.concatenate([source_1_A, source_2_A], axis=1)
        log_likelihood = tfd.Normal(loc = (jnp.matmul(A,jnp.array([s1,s2]).reshape(-1,1)) + jnp.repeat(initial_betas.mean(), (number_time_steps*6)+number_time_steps).reshape(-1,1)), \
                                    scale= jnp.sqrt(sgm)).log_prob(data)

    return log_posterior

param_ranges = []
param_ranges.append((np.maximum(0, rate_1 - emission_granulaty_in_kg_per_s), rate_1 + emission_granulaty_in_kg_per_s))
param_ranges.append((np.maximum(0, rate_2 - emission_granulaty_in_kg_per_s), rate_2 + emission_granulaty_in_kg_per_s))
param_ranges.append((loc_x_1 - grid.dx, loc_x_1 + grid.dx))
param_ranges.append((loc_x_2 - grid.dx, loc_x_2 + grid.dx))
param_ranges.append((loc_y_1 - grid.dy, loc_y_1 + grid.dy))
param_ranges.append((loc_y_2 - grid.dy, loc_y_2 + grid.dy))
param_ranges.append((0, 1e-4))
param_ranges.append((0.5, 1.2))
param_ranges.append((0.5, 1.2))
param_ranges.append((0.5, 1.01))
param_ranges.append((0.5, 1.01))


num_samples = 50_000
lh_samples = latin_hypercube_sampling(param_ranges, num_samples)
# Step function
def lh_one_step(_, parameters):
    rate_1, rate_2, loc_x_1, loc_x_2, loc_y_1, loc_y_2, sgm, a_h, a_v, b_h, b_v = parameters
    new_likelihood = latin_hypercube_likelihood(rate_1, rate_2, loc_x_1, loc_x_2, loc_y_1, loc_y_2, sgm, a_h, a_v, b_h, b_v, "Draxler", False, wind_field.shape[0])
    return new_likelihood, new_likelihood
# Use lax.scan to iterate over the parameter combinations
_, lh_likelihoods = lax.scan(lh_one_step, 0.0, lh_samples)

# Analysing output
lh_likelihood_df = pd.DataFrame(lh_likelihoods, columns = ['log_likelihood'])
lh_parameters_df = pd.DataFrame(lh_samples, columns = ['s_1', 's_2', 'x_1', 'x_2', 'y_1', 'y_2', 'sgm', 'a_h', 'a_v', 'b_h', 'b_v'])
lh_max_likelihood_index = lh_likelihood_df.idxmax()
lh_max_likelihood_row = lh_parameters_df.iloc[lh_max_likelihood_index]
print(f"Latin Hypercube maximum likelihood estimation:")
lh_max_likelihood_row

Latin Hypercube maximum likelihood estimation:


,s_1,s_2,x_1,x_2,y_1,y_2,sgm,a_h,a_v,b_h,b_v
10492,0.000544,0.000154,74.328962,70.322401,44.000383,100.905596,0.000095,1.138209,1.141391,0.959233,0.907669


<br>

### 3.3 Manifold-Metropolis-within-Gibbs

Here we estimate all parameters simultaneously.

In [16]:
initial_log_rates = jnp.log(lh_max_likelihood_row[['s_' + str(i+1) for i in range(int(len(ranges)/3)*2)]].values.flatten())
initial_locations_x = lh_max_likelihood_row[['x_' + str(i+1) for i in range(int(len(ranges)/3)*2)]].values.flatten()
initial_locations_y = lh_max_likelihood_row[['y_' + str(i+1) for i in range(int(len(ranges)/3)*2)]].values.flatten()
initial_sgm_sqr = lh_max_likelihood_row.values.flatten()[6]
initial_log_a_h = jnp.log(lh_max_likelihood_row.values.flatten()[7])
initial_log_a_v = jnp.log(lh_max_likelihood_row.values.flatten()[8])
initial_log_b_h = jnp.log(lh_max_likelihood_row.values.flatten()[9])
initial_log_b_v = jnp.log(lh_max_likelihood_row.values.flatten()[10])


print(f"Setting Initial Parameter Values:")
print(f"--------------------------------")
print(f"Initial rates: {initial_log_rates}")
print(f"Initial locations x: {initial_locations_x}")
print(f"Initial locations y: {initial_locations_y}")
print(f"Initial sgm sqr: {initial_sgm_sqr}")
print(f"Initial a_h: {initial_log_a_h}")
print(f"Initial a_v: {initial_log_a_v}")
print(f"Initial b_h: {initial_log_b_h}")
print(f"Initial b_v: {initial_log_b_v}")

Setting Initial Parameter Values:
--------------------------------
Initial rates: [-7.51653702 -8.77812005]
Initial locations x: [74.32896237 70.32240054]
Initial locations y: [ 44.00038264 100.90559624]
Initial sgm sqr: 9.534536963112652e-05
Initial a_h: 0.12945601579621763
Initial a_v: 0.1322478785039079
Initial b_h: -0.04162160332308528
Initial b_v: -0.0968756455617121


In [17]:
Gibbsparams = {
    'background': initial_betas,
    'sigma_squared':   initial_sgm_sqr,
}
gibbs_flat, gibbs_unflat_func = ravel_pytree(Gibbsparams)

MHparams = {
    'log_a_H': initial_log_a_h,
    'log_a_V': initial_log_a_v,
    'log_b_H':  initial_log_b_h,
    'log_b_V': initial_log_b_v,
    'log_s': jnp.array(initial_log_rates),
    'source_x': jnp.array(initial_locations_x),
    'source_y': jnp.array(initial_locations_y),
}
mh_flat, mh_unflat_func = ravel_pytree(MHparams)


def log_posterior(params, sigma_squared, betas, ss_var, ss_mean, data, priors, wind_sigma, number_of_time_steps, release_17=False):
    """
    Returns the positive log posterior of the point sensors measurements model. 

    """
    if wind_sigma == True:
        coupling_matrix_ref1 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref1, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref2 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref2, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref3 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref3, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref4 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref4, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref5 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref5, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref6 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref6, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref7 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref7, params["source_x"], params["source_y"], None, jnp.exp(params["log_a_H"]), jnp.exp(params["log_a_V"]), jnp.exp(params["log_b_H"]), jnp.exp(params["log_b_V"]), simulation=False, estimated=True, scheme="Draxler", stability_class="D")
    elif wind_sigma == False:
        coupling_matrix_ref1 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref1,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref2 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref2,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref3 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref3,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref4 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref4,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref5 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref5,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref6 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref6,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
        coupling_matrix_ref7 = gaussianplume.temporal_gridfree_coupling_matrix(fixed_ref7,params["source_x"],params["source_y"], None, False, False, False, False, simulation=False, estimated=True, scheme="Draxler", stability_class="D")
    if release_17:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_of_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_of_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_of_time_steps - 31,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_of_time_steps - 31,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_of_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_of_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_of_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_of_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_of_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_of_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_of_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_of_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_of_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_of_time_steps,17*5, order='F')
    else:
        reshaped_coupling_matrix_ref1_src1 = coupling_matrix_ref1[:,0].reshape(number_of_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref1_src2 = coupling_matrix_ref1[:,1].reshape(number_of_time_steps,18*5, order='F')
        reshaped_coupling_matrix_ref2_src1 = coupling_matrix_ref2[:,0].reshape(number_of_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref2_src2 = coupling_matrix_ref2[:,1].reshape(number_of_time_steps,33*5, order='F')
        reshaped_coupling_matrix_ref3_src1 = coupling_matrix_ref3[:,0].reshape(number_of_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref3_src2 = coupling_matrix_ref3[:,1].reshape(number_of_time_steps,22*5, order='F')
        reshaped_coupling_matrix_ref4_src1 = coupling_matrix_ref4[:,0].reshape(number_of_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref4_src2 = coupling_matrix_ref4[:,1].reshape(number_of_time_steps,49*5, order='F')
        reshaped_coupling_matrix_ref5_src1 = coupling_matrix_ref5[:,0].reshape(number_of_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref5_src2 = coupling_matrix_ref5[:,1].reshape(number_of_time_steps,42*5, order='F')
        reshaped_coupling_matrix_ref6_src1 = coupling_matrix_ref6[:,0].reshape(number_of_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref6_src2 = coupling_matrix_ref6[:,1].reshape(number_of_time_steps,29*5, order='F')
        reshaped_coupling_matrix_ref7_src1 = coupling_matrix_ref7[:,0].reshape(number_of_time_steps,17*5, order='F')
        reshaped_coupling_matrix_ref7_src2 = coupling_matrix_ref7[:,1].reshape(number_of_time_steps,17*5, order='F')

    path_averaged_coupling_matrix_ref1_src1 = reshaped_coupling_matrix_ref1_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref1_src2 = reshaped_coupling_matrix_ref1_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src1 = reshaped_coupling_matrix_ref2_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref2_src2 = reshaped_coupling_matrix_ref2_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src1 = reshaped_coupling_matrix_ref3_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref3_src2 = reshaped_coupling_matrix_ref3_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src1 = reshaped_coupling_matrix_ref4_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref4_src2 = reshaped_coupling_matrix_ref4_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src1 = reshaped_coupling_matrix_ref5_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref5_src2 = reshaped_coupling_matrix_ref5_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src1 = reshaped_coupling_matrix_ref6_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref6_src2 = reshaped_coupling_matrix_ref6_src2.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src1 = reshaped_coupling_matrix_ref7_src1.mean(axis=1)
    path_averaged_coupling_matrix_ref7_src2 = reshaped_coupling_matrix_ref7_src2.mean(axis=1)
    
    source_1_path_averaged_A = [path_averaged_coupling_matrix_ref1_src1, path_averaged_coupling_matrix_ref2_src1, path_averaged_coupling_matrix_ref3_src1, path_averaged_coupling_matrix_ref4_src1, path_averaged_coupling_matrix_ref5_src1, path_averaged_coupling_matrix_ref6_src1, path_averaged_coupling_matrix_ref7_src1]
    source_2_path_averaged_A = [path_averaged_coupling_matrix_ref1_src2, path_averaged_coupling_matrix_ref2_src2, path_averaged_coupling_matrix_ref3_src2, path_averaged_coupling_matrix_ref4_src2, path_averaged_coupling_matrix_ref5_src2, path_averaged_coupling_matrix_ref6_src2, path_averaged_coupling_matrix_ref7_src2]
    source_1_A = jnp.array(source_1_path_averaged_A).reshape(-1,1)
    source_2_A = jnp.array(source_2_path_averaged_A).reshape(-1,1)

    A = jnp.concatenate([source_1_A, source_2_A], axis=1)

    log_likelihood = tfd.Normal(loc = (jnp.matmul(A,jnp.exp(params["log_s"]).reshape(-1,1))+ betas), \
                                scale= jnp.sqrt(sigma_squared)).log_prob(data)

    if wind_sigma == True:
        log_prior_a_H = tfd.Normal(loc = priors.a_mean, scale = jnp.sqrt(priors.a_var)).log_prob(params["log_a_H"])
        log_prior_a_V = tfd.Normal(loc = priors.a_mean, scale = jnp.sqrt(priors.a_var)).log_prob(params["log_a_V"])
        log_prior_b_H = tfd.Normal(loc = priors.b_mean, scale = jnp.sqrt(priors.b_var)).log_prob(params["log_b_H"])
        log_prior_b_V = tfd.Normal(loc = priors.b_mean, scale = jnp.sqrt(priors.b_var)).log_prob(params["log_b_V"])

    log_posterior_emission_rate = tfd.MultivariateNormalDiag(loc = ss_mean, scale_diag = jnp.sqrt(ss_var)).log_prob(params["log_s"].reshape(-1,1))
    log_posterior_source_location = tfd.MultivariateNormalDiag(loc = jnp.array([priors.source_location_x_mean, priors.source_location_x_mean, priors.source_location_y_mean, priors.source_location_y_mean]), \
                                                                scale_diag = jnp.sqrt(jnp.array([priors.source_location_x_var, priors.source_location_x_var, priors.source_location_y_var, priors.source_location_y_var]))).log_prob(jnp.array([params["source_x"], params["source_y"]]).flatten())

    if wind_sigma == True:
        log_posterior = jnp.sum(log_likelihood) + jnp.sum(log_posterior_source_location) + jnp.sum(log_posterior_emission_rate) + jnp.sum(log_prior_a_H) + jnp.sum(log_prior_a_V) + jnp.sum(log_prior_b_H) + jnp.sum(log_prior_b_V) 
    elif wind_sigma == False:
        log_posterior = jnp.sum(log_likelihood) + jnp.sum(log_posterior_source_location) + jnp.sum(log_posterior_emission_rate)

    return log_posterior, A



In [18]:
iterations = 50_000
initial_step_size = 0.01
# Run the MCMC algorithm
mala_chains = mcmc.Manifold_MALA_Within_Gibbs(False, gaussianplume, data, log_posterior, priors, MHparams, Gibbsparams, fixed, chilbolton=True, wind_sigmas=True, release_17 = False, step_size_tuning="False").manifold_mala_chains(Gibbsparams, mh_flat, iterations, initial_step_size, release_17=False)


2024-07-23 14:23:02.374332: E external/xla/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_scan] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2024-07-23 14:23:51.100855: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 2m48.726577964s

********************************
[Compiling module jit_scan] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************


Running time Manifold MALA within Gibbs: 1631 minutes 11 seconds
